In [1]:
import re
import os
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import urllib 

# 解析

In [66]:
def recursive_parser1(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
#             print(tag)
#             print(tag.attrs)
            
            content = child.text.strip()
#             print(content)
            if len(content) == 0:
#                 print("000")
                continue
            
            if tag == 'ul' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['vitTOC']:
                    continue
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['mediaPrmo ss']:
                    continue
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['credits']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe', 'em', 'figure', 'aside']: # 加视频
#                 print("111")
                continue

            if tag == 'center':
                texts.append(content)
                continue
            


            if tag == 'div' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['printpreview_header', 'fdaWrapper', 'adCont']:
                    continue
            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
                    
            if tag == 'section' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                class_ = child.attrs['class'][0]
                if class_ in ['editorial-sources']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
#                 print(class_)
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'horizontal-promo__container', 'reviewed-tooltip reviewed-tooltip--medicallyreviewed',#'eh-widget', 
                              'eh-widget eh-widget--horizontal-promo', 'editorial-sources__title', 
                              'feedier-container','editorial-sources__content'
                              'eh-widget eh-widget--latest-feed', 'photogallery-item__media-info'
                             ]:
#                     print("1")
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
#                     print("2")
                    recursive_parser1(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol']:
                # 这几类标签 直接递归处理子标签
                recursive_parser1(child, texts, '')
                
                
            else:
                # 剩余情况直接获取标签内容
#                 print("888")
                texts.append(content)
                
#             print(texts)
        except Exception as e:
            print(e)

In [24]:
def recursive_parser2(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
    
            
            content = child.text.strip()
            if len(content) == 0:
                continue
            
            if tag == 'ul' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['vitTOC']:
                    continue
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['mediaPrmo ss']:
                    continue
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['credits']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe', 'em']: # 加视频
                continue

            if tag == 'center':
                texts.append(content)
                continue
            
#             if tag == 'aside':
#                 continue

            if tag == 'div' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['printpreview_header', 'fdaWrapper', 'adCont']:
                    continue
            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'drug-headline__subheader', 'eh-widget eh-widget--byline', 'ad-block ad-block--boxtop _hold-size', 
                              'eh-widget eh-widget--ad', 'drug-widget drug-rating', 'drug-widget drug-images', 
                              'otherNames', 'apPageBottom', 'contentad', 'medianet', 'from_webmd', 'toc_lite', 'monolastreviewed', 'drug_rel_main', 'disclaimer']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser2(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser2(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [68]:
def parser(input_path, output_path, tag, results, mislist):
    for root, dirs, file_names in os.walk(input_path):
        """
        os.walk()
        root 所指的是当前正在遍历的这个文件夹的本身的地址
        files_names同样是 list , 内容是该文件夹中所有的文件(不包括子目录)
        函数会自动改变root的值使得遍历所有的子文件夹。
        所以返回的三元元组的个数为所有子文件夹（包括子子文件夹，子子子文件夹等等）加上1（根文件夹）
        """
        
        for file_name in file_names:
            html_file = os.path.join(root, file_name)
            html_content = open(html_file, 'r', encoding='utf-8').read()
            selector = Selector(text=html_content)
            try:
                title = selector.xpath("//h1//text()").extract()[0].strip()
                print(html_file)
            except Exception as e:
                print(e)
#                 print(html_file)
                continue

            texts = []
            
            soup = BeautifulSoup(html_content, 'html.parser') 

            if tag=='health':
                item = soup.find('div',{'class':'col-lg-8 article-template__main'})
                # print(item)
                if item is None:
                    item = soup.find('div',{'class':'col-lg-8 photogallery--items-col'})
                        
            if tag=='drugs':
                item = soup.find('div',{'class':'drugs-template__container'})
                
            if item is None:
                # print(html_file)
                mislist.append(html_file)
                continue
            else:
                # print(html_file)
                if tag=='health':
                    recursive_parser1(item, texts, "")
                if tag=='drugs':
                    recursive_parser2(item, texts, "")  
                content = '\n'.join(texts)
                if content == "":
                    continue
                tmp = {
                    "title": title,
                    "content": content
                }
#                 print(tmp)
                results.append(tmp)

In [11]:
tags = ['health', 'drugs']
output_path = 'D:/Intern_Project/5_everydayhealth/'
for tag in tags: 
    results = []
    print(tag)
    source_path = f'D:/Intern_Project/5_everydayhealth/{tag}_html'
    parser(source_path, output_path, tag, results)
    with open(f'D:/Intern_Project/5_everydayhealth/{tag}_parser_results.json', 'w', encoding="utf-8") as f:
        f.write(json.dumps(results, indent=1, ensure_ascii=False))
        # f.write('\n'.join(results))

health
D:/Intern_Project/5_everydayhealth/health_html\Addiction.html
D:/Intern_Project/5_everydayhealth/health_html\Alternative Health.html
D:/Intern_Project/5_everydayhealth/health_html\Anatomy.html
D:/Intern_Project/5_everydayhealth/health_html\Anthrax.html
D:/Intern_Project/5_everydayhealth/health_html\Arthritis.html
D:/Intern_Project/5_everydayhealth/health_html\Autoimmune Diseases.html
D:/Intern_Project/5_everydayhealth/health_html\Back Pain.html
D:/Intern_Project/5_everydayhealth/health_html\Beauty Toolbox.html
D:/Intern_Project/5_everydayhealth/health_html\Black Health.html
D:/Intern_Project/5_everydayhealth/health_html\black-health-facts--statement.html
D:/Intern_Project/5_everydayhealth/health_html\Blood Disorders.html
D:/Intern_Project/5_everydayhealth/health_html\Bone Cancer.html
D:/Intern_Project/5_everydayhealth/health_html\Bone Fractures.html
D:/Intern_Project/5_everydayhealth/health_html\Brain Cancer.html
D:/Intern_Project/5_everydayhealth/health_html\Chronic Fatigue Syn

D:/Intern_Project/5_everydayhealth/health_html\Infectious Diseases.html
D:/Intern_Project/5_everydayhealth/health_html\Iron Deficiency Anemia.html
D:/Intern_Project/5_everydayhealth/health_html\Irritated Skin.html
D:/Intern_Project/5_everydayhealth/health_html\Kidney Cancer.html
D:/Intern_Project/5_everydayhealth/health_html\Kids' Health.html
D:/Intern_Project/5_everydayhealth/health_html\Knee Pain.html
D:/Intern_Project/5_everydayhealth/health_html\Liver Cancer.html
D:/Intern_Project/5_everydayhealth/health_html\Longevity.html
D:/Intern_Project/5_everydayhealth/health_html\Lung & Respiratory.html
D:/Intern_Project/5_everydayhealth/health_html\Macular Degeneration.html
D:/Intern_Project/5_everydayhealth/health_html\Men's Health.html
D:/Intern_Project/5_everydayhealth/health_html\migraine--consumers-guide-to-cgrp-medications-for-chronic-migraine.html
D:/Intern_Project/5_everydayhealth/health_html\multiple-myeloma--disparities-in-care-for-black-people-with-multiple-myeloma.html
D:/Intern

In [22]:
print(len(results))

13773


In [16]:
for item in results:
    item['content'] = item['content'].split("OVERDOSE SIGNS")[0].split("Overdose Signs")[0]

In [12]:
results[3]

IndexError: list index out of range

In [21]:
with open(f'D:/Intern_Project/5_everydayhealth/drugs_parser_results.json', 'w', encoding="utf-8") as f:
        f.write(json.dumps(results, indent=1, ensure_ascii=False))

In [69]:
# 再单独跑一遍health的
mislist = []
results = []
tag = 'health'
output_path = 'D:/Intern_Project/5_everydayhealth/'
source_path = f'D:/Intern_Project/5_everydayhealth/{tag}_html'
parser(source_path, output_path, tag, results, mislist)

D:/Intern_Project/5_everydayhealth/health_html\Abdominal Pain.html
D:/Intern_Project/5_everydayhealth/health_html\Abortion.html
D:/Intern_Project/5_everydayhealth/health_html\ACE Inhibitors.html
D:/Intern_Project/5_everydayhealth/health_html\Acetylcholine.html
D:/Intern_Project/5_everydayhealth/health_html\acinetobacter--guide.html
D:/Intern_Project/5_everydayhealth/health_html\Acinetobacter.html
D:/Intern_Project/5_everydayhealth/health_html\ACL Tear.html
D:/Intern_Project/5_everydayhealth/health_html\acne--new-findings-support-the-rising-popularity-of-spironolactone-for-adult-acne.html
D:/Intern_Project/5_everydayhealth/health_html\acne.aspx.html
D:/Intern_Project/5_everydayhealth/health_html\Acne.html
D:/Intern_Project/5_everydayhealth/health_html\addiction--is-ozempic-really-an-anti-addiction-drug.html
D:/Intern_Project/5_everydayhealth/health_html\Addiction.html
D:/Intern_Project/5_everydayhealth/health_html\Adenoidectomy.html
D:/Intern_Project/5_everydayhealth/health_html\adhd--s

D:/Intern_Project/5_everydayhealth/health_html\back-pain--spinal-cord-stimulators-for-chronic-back-pain-win-fda-approval.html
D:/Intern_Project/5_everydayhealth/health_html\Bacterial Vaginosis.html
D:/Intern_Project/5_everydayhealth/health_html\Barbiturates.html
D:/Intern_Project/5_everydayhealth/health_html\Barrett's Esophagus.html
D:/Intern_Project/5_everydayhealth/health_html\Beauty Toolbox.html
D:/Intern_Project/5_everydayhealth/health_html\Bedbugs.html
D:/Intern_Project/5_everydayhealth/health_html\Bedwetting.html
D:/Intern_Project/5_everydayhealth/health_html\Benign Prostatic Hyperplasia.html
D:/Intern_Project/5_everydayhealth/health_html\Benzodiazepines.html
D:/Intern_Project/5_everydayhealth/health_html\Beta Blockers.html
D:/Intern_Project/5_everydayhealth/health_html\Binge Eating Disorder.html
D:/Intern_Project/5_everydayhealth/health_html\binge-eating-disorder--guide.html
D:/Intern_Project/5_everydayhealth/health_html\biopsy--guide.html
D:/Intern_Project/5_everydayhealth/heal

D:/Intern_Project/5_everydayhealth/health_html\C. Difficile.html
D:/Intern_Project/5_everydayhealth/health_html\Calcium Channel Blockers.html
D:/Intern_Project/5_everydayhealth/health_html\Calories.html
D:/Intern_Project/5_everydayhealth/health_html\Campylobacter.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--avoid-infections-during-chemotherapy.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--black-people-with-lung-cancer-can-find-community-in-support-groups.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--body-image-after-cancer-expert-tips-for-dealing-with-insecurities.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--breast-cancer--top-6-hazards-breast-implant-surgery.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--can-exercise-prevent-some-types-cancer.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--cancers-that-affect-asian-americans-the-most.html
D:/Intern_Project/5_everydayhealth/health_html\cancer--cancers-th

D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--a-day-in-the-life-of-someone-with-crohns-disease.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--an-unlikely-friendship-blossoms-into-a-caregiving-relationship.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--are-the-headlines-about-ai-changing-the-world-stressing-you-out.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--caregivers-for-best-results-train-them-and-set-expectations.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--depression-made-me-fat-but-ozempic-made-me-depressed.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--good-grief-what-does-it-mean-to-grieve-well.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--how-to-navigate-travel-as-a-blood-clot-survivor.html
D:/Intern_Project/5_everydayhealth/health_html\columns--my-health-story--

D:/Intern_Project/5_everydayhealth/health_html\coronary-heart-disease--american-heart-association-releases-first-guideline-overhaul-in-a-decade.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--all-your-covid-vaccine-questions-answered.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--biggest-covid-vaccine-myths.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--diabetes-drug-metformin-may-reduce-long-covid-risk.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--does-covid-19-wreak-havoc-with-the-menstrual-cycle.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--eu-1-1-emerges-as-the-latest-covid-19-subvariant.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--fda-authorizes-second-omicron-booster-for-seniors-and-immunocompromised.html
D:/Intern_Project/5_everydayhealth/health_html\coronavirus--hearing-loss-and-tinnitus-are-lesser-known-symptoms-of-covid-19.html
D:/Intern_Project/5_everydayhealth/heal

...

D:/Intern_Project/5_everydayhealth/health_html\psoriasis--when-topical-treatments-are-not-enough.html
D:/Intern_Project/5_everydayhealth/health_html\psoriasis--why-maintaining-a-healthy-weight-can-help-psoriasis.html
D:/Intern_Project/5_everydayhealth/health_html\Psoriasis.html
D:/Intern_Project/5_everydayhealth/health_html\Psoriatic Arthritis.html
D:/Intern_Project/5_everydayhealth/health_html\psoriatic-arthritis--aging.html
D:/Intern_Project/5_everydayhealth/health_html\psoriatic-arthritis--consumers-guide-to-jak-inhibitors.html
D:/Intern_Project/5_everydayhealth/health_html\psoriatic-arthritis--easy-travel-tips.html
D:/Intern_Project/5_everydayhealth/health_html\psoriatic-arthritis--how-psoriatic-arthritis-is-linked-to-psoriasis.html
D:/Intern_Project/5_everydayhealth/health_html\psoriatic-arthritis--living-with--psoriatic-arthritis-avoid-infections.html
D:/Intern_Project/5_everydayhealth/health_html\psoriatic-arthritis--living-with--top-psoriatic-arthritis-triggers.html
D:/Intern_P

D:/Intern_Project/5_everydayhealth/health_html\Scarlet Fever.html
D:/Intern_Project/5_everydayhealth/health_html\schizophrenia--could-turmeric-help-people-living-with-schizophrenia.html
D:/Intern_Project/5_everydayhealth/health_html\schizophrenia--glossary.html
D:/Intern_Project/5_everydayhealth/health_html\schizophrenia--guide--schizoaffective.html
D:/Intern_Project/5_everydayhealth/health_html\schizophrenia--guide--symptoms.html
D:/Intern_Project/5_everydayhealth/health_html\Schizophrenia.html
D:/Intern_Project/5_everydayhealth/health_html\Sciatica.html
D:/Intern_Project/5_everydayhealth/health_html\Seasonal Allergies.html
D:/Intern_Project/5_everydayhealth/health_html\Seasonal Skin Guide.html
D:/Intern_Project/5_everydayhealth/health_html\Sebaceous Cyst.html
D:/Intern_Project/5_everydayhealth/health_html\Sedatives.html
D:/Intern_Project/5_everydayhealth/health_html\self-care--alcohol-free-ways-to-unwind-at-the-end-of-a-long-day.html
D:/Intern_Project/5_everydayhealth/health_html\sel

D:/Intern_Project/5_everydayhealth/health_html\SNRIs.html
D:/Intern_Project/5_everydayhealth/health_html\Social Anxiety Disorder.html
D:/Intern_Project/5_everydayhealth/health_html\Sore Throat.html
D:/Intern_Project/5_everydayhealth/health_html\specialists--cancer--cll-health-complications.html
D:/Intern_Project/5_everydayhealth/health_html\SSRIs.html
D:/Intern_Project/5_everydayhealth/health_html\Statins.html
D:/Intern_Project/5_everydayhealth/health_html\STDs.html
D:/Intern_Project/5_everydayhealth/health_html\Stent.html
D:/Intern_Project/5_everydayhealth/health_html\Stomach cancer.html
D:/Intern_Project/5_everydayhealth/health_html\Stop Smoking.html
D:/Intern_Project/5_everydayhealth/health_html\stop-smoking-celebrities-pictures.html
D:/Intern_Project/5_everydayhealth/health_html\stop-smoking-pictures--the-best-and-worst-ways-to-quit-smoking.aspx.html
D:/Intern_Project/5_everydayhealth/health_html\Strabismus.html
D:/Intern_Project/5_everydayhealth/health_html\streptococcus--strep-th

D:/Intern_Project/5_everydayhealth/health_html\type-2-diabetes--symptoms--warning-signs-of-type-2-diabetes.html
D:/Intern_Project/5_everydayhealth/health_html\type-2-diabetes--white-rice-bacon-and-other-processed-foods-driving-up-type-2-diabetes-rates.html
D:/Intern_Project/5_everydayhealth/health_html\Ulcer.html
D:/Intern_Project/5_everydayhealth/health_html\Ulcerative Colitis.html
D:/Intern_Project/5_everydayhealth/health_html\ulcerative-colitis--are-your-ulcerative-colitis-symptoms-under-control.html
D:/Intern_Project/5_everydayhealth/health_html\ulcerative-colitis--check-in-check-up.html
D:/Intern_Project/5_everydayhealth/health_html\ulcerative-colitis--diet--summer-treats-people-with-uc.html
D:/Intern_Project/5_everydayhealth/health_html\ulcerative-colitis--guide--diet.html
D:/Intern_Project/5_everydayhealth/health_html\ulcerative-colitis--how-aerobic-exercise-can-help-ulcerative-colitis.aspx.html
D:/Intern_Project/5_everydayhealth/health_html\ulcerative-colitis--how-good-is-your-

D:/Intern_Project/5_everydayhealth/health_html\weight--here-is-when-doctors-say-to-consider-a-weight-loss-drug-like-ozempic.html
D:/Intern_Project/5_everydayhealth/health_html\weight--intermittent-fasting-about-equal-to-calorie-counting-for-weight-loss.html
D:/Intern_Project/5_everydayhealth/health_html\weight--is-it-safe-to-use-compounded-semaglutide-and-tirzepatide-for-weight-loss.html
D:/Intern_Project/5_everydayhealth/health_html\weight--prescription-weight-loss-drugs-ozempic-wegovy-mounjaro-saxenda-and-others.html
D:/Intern_Project/5_everydayhealth/health_html\weight--risks-you-should-avoid-when-buying-weight-loss-drugs-online.html
D:/Intern_Project/5_everydayhealth/health_html\weight--semaglutide-relieves-my-obsession-with-food.html
D:/Intern_Project/5_everydayhealth/health_html\weight--type-2-diabetes-drug-mounjaro-leads-to-weight-loss-in-new-trial-for-treatment-of-overweight-or-obesity.html
D:/Intern_Project/5_everydayhealth/health_html\weight--wegovy-can-reduce-heart-attack-an

In [70]:
with open(f'D:/Intern_Project/5_everydayhealth/{tag}_parser_results.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(results, indent=1, ensure_ascii=False))

In [74]:
for item in results:
    item['content'] = item['content'].split("Additional reporting")[0]

In [57]:
print(len(mislist))

160


In [71]:
print(len(results))

1839


In [76]:
with open(f'D:/Intern_Project/5_everydayhealth/{tag}_parser_results.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(results, indent=1, ensure_ascii=False))

In [ ]:
def parser2(mislist, results):
    for html_file in mislist:
        html_content = open(html_file, 'r', encoding='utf-8').read()
        selector = Selector(text=html_content)
        try:
            title = selector.xpath("//h1//text()").extract()[0].strip()
        except Exception as e:
            print(e)
            print(html_file)
            continue

        texts = []

        soup = BeautifulSoup(html_content, 'html.parser') 

        item = soup.find('article',{'class':'ift-template'})

        if item is None:
            print(html_file)
            continue
        else:
            recursive_parser3(item, texts, "")  
            content = '\n'.join(texts)
            if content == "":
                continue
            tmp = {
                "title": title,
                "content": content
            }
            results.append(tmp)

In [ ]:
# 单独解析mis掉的health中的某部分格式
def recursive_parser3(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
            
            content = child.text.strip()
            if len(content) == 0:
                continue
            
            if tag == 'ul' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['vitTOC']:
                    continue
            if tag == 'a' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['mediaPrmo ss']:
                    continue
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['credits']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe', 'em', 'figure']: # 加视频
                continue

            if tag == 'center':
                texts.append(content)
                continue
            
#             if tag == 'aside':
#                 continue

            if tag == 'div' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['printpreview_header', 'fdaWrapper', 'adCont']:
                    continue
            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'sticky-share-bar-container', 'eh-widget eh-widget--hero', 'eh-widget eh-widget--byline', 'eh-widget',
                              'eh-widget eh-widget--anchor-menu','ift-template__bottom'
                              'otherNames', 'apPageBottom', 'contentad', 'medianet', 'from_webmd', 'toc_lite', 'monolastreviewed', 'drug_rel_main', 'disclaimer']:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser2(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol', 'article']:
                # 这几类标签 直接递归处理子标签
                recursive_parser2(child, texts, '')
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)

        except Exception as e:
            print(e)

In [ ]:
results = []
parser2(mislist, results)

In [ ]:
with open(f'D:/Intern_Project/5_everydayhealth/append_parser_results.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(results, indent=1, ensure_ascii=False))